In [1]:
import pandas as pd
import numpy as np
import geopandas as gpp
import seaborn as sns

df_votacao_candidato_munzona = pd.read_csv('/Users/willianribeiro/Documents/GitHub/analise_exploratoria_dados_tse/data/Resultados/2020/votacao_candidato_munzona_2020/votacao_candidato_munzona_2020_BRASIL.csv', sep=';', encoding='Latin 1')

df_detalhe_votacao_munzona = pd.read_csv('/Users/willianribeiro/Documents/GitHub/analise_exploratoria_dados_tse/data/Resultados/2020/detalhe_votacao_munzona_2020/detalhe_votacao_munzona_2020_BRASIL.csv', sep=';', encoding='Latin 1')

In [2]:
df_votacao_candidato_munzona.columns

Index(['DT_GERACAO', 'HH_GERACAO', 'ANO_ELEICAO', 'CD_TIPO_ELEICAO',
       'NM_TIPO_ELEICAO', 'NR_TURNO', 'CD_ELEICAO', 'DS_ELEICAO', 'DT_ELEICAO',
       'TP_ABRANGENCIA', 'SG_UF', 'SG_UE', 'NM_UE', 'CD_MUNICIPIO',
       'NM_MUNICIPIO', 'NR_ZONA', 'CD_CARGO', 'DS_CARGO', 'SQ_CANDIDATO',
       'NR_CANDIDATO', 'NM_CANDIDATO', 'NM_URNA_CANDIDATO',
       'NM_SOCIAL_CANDIDATO', 'CD_SITUACAO_CANDIDATURA',
       'DS_SITUACAO_CANDIDATURA', 'CD_DETALHE_SITUACAO_CAND',
       'DS_DETALHE_SITUACAO_CAND', 'TP_AGREMIACAO', 'NR_PARTIDO', 'SG_PARTIDO',
       'NM_PARTIDO', 'NR_FEDERACAO', 'NM_FEDERACAO', 'SG_FEDERACAO',
       'DS_COMPOSICAO_FEDERACAO', 'SQ_COLIGACAO', 'NM_COLIGACAO',
       'DS_COMPOSICAO_COLIGACAO', 'ST_VOTO_EM_TRANSITO', 'QT_VOTOS_NOMINAIS',
       'NM_TIPO_DESTINACAO_VOTOS', 'QT_VOTOS_NOMINAIS_VALIDOS',
       'CD_SIT_TOT_TURNO', 'DS_SIT_TOT_TURNO'],
      dtype='object')

In [3]:
df_detalhe_votacao_munzona.columns

Index(['DT_GERACAO', 'HH_GERACAO', 'ANO_ELEICAO', 'CD_TIPO_ELEICAO',
       'NM_TIPO_ELEICAO', 'NR_TURNO', 'CD_ELEICAO', 'DS_ELEICAO', 'DT_ELEICAO',
       'TP_ABRANGENCIA', 'SG_UF', 'SG_UE', 'NM_UE', 'CD_MUNICIPIO',
       'NM_MUNICIPIO', 'NR_ZONA', 'CD_CARGO', 'DS_CARGO', 'QT_APTOS',
       'QT_SECOES_PRINCIPAIS', 'QT_SECOES_AGREGADAS',
       'QT_SECOES_NAO_INSTALADAS', 'QT_TOTAL_SECOES', 'QT_COMPARECIMENTO',
       'QT_ELEITORES_SECOES_NAO_INSTALADAS', 'QT_ABSTENCOES',
       'ST_VOTO_EM_TRANSITO', 'QT_VOTOS', 'QT_VOTOS_CONCORRENTES',
       'QT_VOTOS_VALIDOS', 'QT_VOTOS_NOMINAIS_VALIDOS',
       'QT_TOTAL_VOTOS_LEG_VALIDOS', 'QT_VOTOS_LEGENDA_VALIDOS',
       'QT_VOTOS_NOMINAIS_CONVR_LEG', 'QT_TOTAL_VOTOS_ANULADOS',
       'QT_VOTOS_NOMINAIS_ANULADOS', 'QT_VOTOS_LEGENDA_ANULADOS',
       'QT_TOTAL_VOTOS_ANUL_SUBJUD', 'QT_VOTOS_NOMINAIS_ANUL_SUBJUD',
       'QT_VOTOS_LEGENDA_ANUL_SUBJUD', 'QT_VOTOS_BRANCOS',
       'QT_TOTAL_VOTOS_NULOS', 'QT_VOTOS_NULOS', 'QT_VOTOS_NULO_TECNICO',


In [10]:
## Criação do dicionario de cidades primeiramente

def dataframe_cidades(dataframe):
    # Lista para armazenar as informações das cidades
    cities_data = []

    # Conjunto para armazenar os códigos de cidades já encontrados
    cities_validator = set()

    # Iterando pelas linhas do dataframe
    for index, row in dataframe.iterrows():
        # Verificando se o código da cidade já foi adicionado
        if row['CD_MUNICIPIO'] not in cities_validator:
            # Adicionando o código da cidade ao conjunto para controle
            cities_validator.add(row['CD_MUNICIPIO'])
            # Adicionando as informações da cidade à lista
            cities_data.append({
                'codigo_cidade': row['CD_MUNICIPIO'],
                'nome_cidade': row['NM_MUNICIPIO'],
                'sigla_estado': row['SG_UF']
            })

    # Criando um DataFrame com as informações coletadas
    dataframe_tratado = pd.DataFrame(cities_data)

    return dataframe_tratado

In [11]:
dataset_cidades = dataframe_cidades(df_votacao_candidato_munzona)

dataset_cidades.head()

,codigo_cidade,nome_cidade,sigla_estado
0,74543,MAUÁ DA SERRA,PR
1,74012,ABATIÁ,PR
2,27855,MACEIÓ,AL
3,88552,SANTO ANTÔNIO DA PATRULHA,RS
4,2135,BENJAMIN CONSTANT,AM


In [21]:
'''
Aqui estou criando parâmetros por cidade sendo eles:
- Quociente Eleitoral
- Número de Eleitores Aptos 'QT_APTOS'
- Quantidade de eleitores que compareceram 'QT_COMPARECIMENTO'
- Quantidade de Abstenções 'QT_ABSTENCOES'
- Quantidade Total de Votos 'QT_VOTOS'
- Quantidade de Votos Validos 'QT_VOTOS_VALIDOS'
- Quantidade total de votos anulados 'QT_TOTAL_VOTOS_ANULADOS'
- Quantidade de votos em branco 'QT_VOTOS_BRANCOS'
- Quantidade de votos nulos 'QT_VOTOS_NULOS'
'''

def somatorio_parametros_por_cidade(dataset_cidades, dataset_votos):
    dataset_cidades_acumulados = dataset_cidades.copy()

    dataset_resultados = dataset_votos[dataset_votos['DS_CARGO'] == 'Vereador']

    # Inicializando as colunas no DataFrame
    colunas = ['quantidade_eleitores_aptos', 'quantidade_eleitores_comparecimento', 'quantidade_abstencoes', 
               'quantidade_total_votos', 'quantidade_votos_validos', 'quantidade_votos_anulados', 
               'quantidade_votos_brancos', 'quantidade_votos_nulos']
    for coluna in colunas:
        dataset_cidades_acumulados[coluna] = 0

    # Iterando pelas linhas do dataframe
    for index, row in dataset_cidades_acumulados.iterrows():
        # Isolando a cidade num dataframe para reduzir o consumo de armazenamento
        dataset_resultados_iteracao = dataset_resultados[dataset_resultados['CD_MUNICIPIO'] == row['codigo_cidade']]

        # Atualizando os valores para a cidade atual
        dataset_cidades_acumulados.at[index, 'quantidade_eleitores_aptos'] = dataset_resultados_iteracao['QT_APTOS'].sum()
        dataset_cidades_acumulados.at[index, 'quantidade_eleitores_comparecimento'] = dataset_resultados_iteracao['QT_COMPARECIMENTO'].sum()
        dataset_cidades_acumulados.at[index, 'quantidade_abstencoes'] = dataset_resultados_iteracao['QT_ABSTENCOES'].sum()
        dataset_cidades_acumulados.at[index, 'quantidade_total_votos'] = dataset_resultados_iteracao['QT_VOTOS'].sum()
        dataset_cidades_acumulados.at[index, 'quantidade_votos_validos'] = dataset_resultados_iteracao['QT_VOTOS_VALIDOS'].sum()
        dataset_cidades_acumulados.at[index, 'quantidade_votos_anulados'] = dataset_resultados_iteracao['QT_TOTAL_VOTOS_ANULADOS'].sum()
        dataset_cidades_acumulados.at[index, 'quantidade_votos_brancos'] = dataset_resultados_iteracao['QT_VOTOS_BRANCOS'].sum()
        dataset_cidades_acumulados.at[index, 'quantidade_votos_nulos'] = dataset_resultados_iteracao['QT_VOTOS_NULOS'].sum()

    return dataset_cidades_acumulados

In [19]:
dataset_extracao_cidades = somatorio_parametros_por_cidade(dataset_cidades=dataset_cidades, dataset_votos=df_detalhe_votacao_munzona)

dataset_extracao_cidades.head()

,codigo_cidade,nome_cidade,sigla_estado,quantidade_eleitores_aptos,quantidade_eleitores_comparecimento,quantidade_abstencoes,quantidade_total_votos,quantidade_votos_validos,quantidade_votos_anulados,quantidade_votos_brancos,quantidade_votos_nulos
0,74543,MAUÁ DA SERRA,PR,7131,5992,1139,5992,5633,0,129,189
1,74012,ABATIÁ,PR,5822,4743,1079,4743,4520,0,103,120
2,27855,MACEIÓ,AL,592388,444070,148318,444070,391530,746,20121,29275
3,88552,SANTO ANTÔNIO DA PATRULHA,RS,33638,25299,8339,25299,23761,0,1052,486
4,2135,BENJAMIN CONSTANT,AM,21269,17248,4021,17248,16877,0,65,306


In [22]:
#dataset_extracao_cidades.to_csv('extracao_cidades.csv', index=False)

In [25]:
coordenadas_municipios = pd.read_csv('https://raw.githubusercontent.com/alanwillms/geoinfo/master/latitude-longitude-cidades.csv', sep=';')

In [30]:
coordenadas_municipios['municipio'] = coordenadas_municipios['municipio'].str.upper()

In [31]:
coordenadas_municipios

,id_municipio,uf,municipio,longitude,latitude
0,2,AC,ACRELÂNDIA,-66.897166,-9.825808
1,3,AC,ASSIS BRASIL,-69.573794,-10.929765
2,4,AC,BRASILÉIA,-68.749696,-10.994994
3,5,AC,BUJARI,-67.955029,-9.815277
4,6,AC,CAPIXABA,-67.686006,-10.566031


In [32]:
# Unindo o dataframe com coordenadas ao dataframe de resultados

dataframe_cidades_merged = pd.merge(dataset_extracao_cidades, coordenadas_municipios, 
                     left_on='nome_cidade', right_on='municipio', 
                     how='left')

In [33]:
dataframe_cidades_merged.head()

,codigo_cidade,nome_cidade,sigla_estado,quantidade_eleitores_aptos,quantidade_eleitores_comparecimento,quantidade_abstencoes,quantidade_total_votos,quantidade_votos_validos,quantidade_votos_anulados,quantidade_votos_brancos,quantidade_votos_nulos,id_municipio,uf,municipio,longitude,latitude
0,74543,MAUÁ DA SERRA,PR,7131,5992,1139,5992,5633,0,129,189,3391.0,PR,MAUÁ DA SERRA,-51.227732,-23.898761
1,74012,ABATIÁ,PR,5822,4743,1079,4743,4520,0,103,120,3173.0,PR,ABATIÁ,-50.313281,-23.304863
2,27855,MACEIÓ,AL,592388,444070,148318,444070,391530,746,20121,29275,70.0,AL,MACEIÓ,-35.734960,-9.665985
3,88552,SANTO ANTÔNIO DA PATRULHA,RS,33638,25299,8339,25299,23761,0,1052,486,4245.0,RS,SANTO ANTÔNIO DA PATRULHA,-50.517489,-29.826768
4,2135,BENJAMIN CONSTANT,AM,21269,17248,4021,17248,16877,0,65,306,134.0,AM,BENJAMIN CONSTANT,-70.034157,-4.377680


In [35]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px

# Suponha que 'dataframe_cidades_merged' é o seu DataFrame
# ... 

# Inicializando a aplicação Dash
app = dash.Dash(__name__)

# Criando uma lista de cidades para a lista suspensa
lista_cidades = dataframe_cidades_merged['nome_cidade'].unique()

# Layout da aplicação
app.layout = html.Div([
    dcc.Dropdown(
        id='cidade-dropdown',
        options=[{'label': cidade, 'value': cidade} for cidade in lista_cidades],
        value=lista_cidades[0]  # Valor padrão
    ),
    dcc.Graph(id='cidade-map')
])

# Callback para atualizar o gráfico baseado na seleção da cidade
@app.callback(
    Output('cidade-map', 'figure'),
    [Input('cidade-dropdown', 'value')]
)
def update_map(selected_city):
    filtered_data = dataframe_cidades_merged[dataframe_cidades_merged['nome_cidade'] == selected_city]
    
    fig = px.scatter_geo(filtered_data,
                         lat='latitude',
                         lon='longitude',
                         hover_name='nome_cidade',
                         hover_data=['quantidade_eleitores_aptos', 'quantidade_eleitores_comparecimento', 'quantidade_abstencoes',
                                     'quantidade_total_votos', 'quantidade_votos_validos', 'quantidade_votos_anulados',
                                     'quantidade_votos_brancos', 'quantidade_votos_nulos'],
                         title=f'Mapa Interativo da Cidade de {selected_city}')
    
    fig.update_layout(geo_scope='south america')
    return fig

# Rodando a aplicação
if __name__ == '__main__':
    app.run_server(debug=True)


/var/folders/hl/kty_dhp50c1c_z0jzx787tl80000gn/T/ipykernel_3938/2638092433.py:2: UserWarning:


The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`

/var/folders/hl/kty_dhp50c1c_z0jzx787tl80000gn/T/ipykernel_3938/2638092433.py:3: UserWarning:


The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`

